In [11]:
import os
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta

from table_wrangle_functions import (
    add_datestamp
)

# Ensure the 'outputs' directory exists
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Load the data
adhd_medication_data = pd.read_csv("/workspaces/ADHD-Pre-and-Post-Covid/output/Patient_table_3_rolling_6month_incident.csv.gz")

# getting the month and year only
adhd_medication_data['last_mph_med_date'] = (
    pd.to_datetime(adhd_medication_data['last_mph_med_date'], format='%Y-%m-%d')
)

adhd_medication_data['last_mph_med_date_month_date'] = (
    adhd_medication_data['last_mph_med_date'].apply(lambda x: x.strftime('%Y-%m'))
)

output = (
    adhd_medication_data
            .groupby(['last_mph_med_date_month_date','age_band','sex'],as_index=False).size()
)

In [37]:
datetime.strptime(min(output['last_mph_med_date_month_date']), '%Y-%m')

datetime.datetime(2016, 3, 1, 0, 0)

In [53]:
from datetime import (
    datetime
)

from dateutil import relativedelta

def monthly_interval_list(start_date, end_date):
    
    #Differnce in months
    delta = relativedelta.relativedelta(end_date, start_date)

    res_months = delta.months + (delta.years * 12)

    interval_array = [start_date + relativedelta.relativedelta(months=each_month) for each_month in range(res_months+1)]

    return interval_array

start_date = datetime.strptime(min(output['last_mph_med_date_month_date']), '%Y-%m')
end_date = datetime.strptime(max(output['last_mph_med_date_month_date']), '%Y-%m')

tmp = monthly_interval_list(start_date,end_date)
print(tmp[0])
print(start_date)
print(tmp[-1])
print(end_date)



2016-03-01 00:00:00
2016-03-01 00:00:00
2025-06-01 00:00:00
2025-06-01 00:00:00


In [14]:
dataframe = output.copy()
date_column = 'last_mph_med_date_month_date'
#date_interval =
column_group = ['age_band','sex']

In [ ]:
unqiue_combination = dataframe[column_group].drop_duplicates(subset=column_group)

#Need to get the vaules of each 
for each_index in unqiue_combination.index:
    
    each_conbination = unqiue_combination.iloc[[each_index],:]

    each_conbination_dict = each_conbination.to_dict('records')[0]

    #Getting the filtered vaules - Source: https://stackoverflow.com/questions/34157811/filter-a-pandas-dataframe-using-values-from-a-dict
    dataframe.loc[(dataframe[list(each_conbination_dict)] == pd.Series(each_conbination_dict)).all(axis=1)]

    #Need to vo


    last_mph_med_date_month_date age_band     sex  size
0                        2016-03   0 to 9  female     1
2                        2016-11   0 to 9  female     1
12                       2018-05   0 to 9  female     1
13                       2018-06   0 to 9  female     1
14                       2018-09   0 to 9  female     1
..                           ...      ...     ...   ...
130                      2025-02   0 to 9  female    23
132                      2025-03   0 to 9  female    21
134                      2025-04   0 to 9  female    36
136                      2025-05   0 to 9  female    36
138                      2025-06   0 to 9  female    45

[72 rows x 4 columns]
    last_mph_med_date_month_date age_band   sex  size
1                        2016-09   0 to 9  male     1
3                        2017-02   0 to 9  male     2
4                        2017-03   0 to 9  male     1
5                        2017-04   0 to 9  male     1
6                        2017-09   